### NID  Neural Interaction Detector

This is on a subset (3000) and run with Comp premium as the response.   

For more info:

https://github.com/mtsang/neural-interaction-detection

Must run this in order. placeholders -> loss function -> session

In [29]:
import numpy as np
import pandas as pd
import math
import bisect
import operator
import tensorflow as tf
from sklearn.preprocessing import StandardScaler


In [30]:
data = pd.read_csv('../data/processed4/7_27_df_model.csv')
# skip all the synthetic stuff. 
data.drop(['TRV.Win'], axis=1, inplace=True)



In [31]:
#np.transpose(data.head())

In [32]:
data.head()

,TRV.Price,tot_bld_annl_rev_amt,fnl_prc_pt_fct,merge_freq,w2v,program_freq,prcnt_blg_tiv,terr_pctl_bldgfire,comp_freq,terr_pctl_liab,...,UWC_DALLAS,csf,market_sh_med,bld_ind,UWC_HARTFORD,afi_pol_cnt,program_rev_BUS PAC NAIL SALONS,LOC_BLD_AGE_bkt_highest,program_rev_GARAGE PAC GENERAL REPAIRS WIT,Competitor.Premium
0,0.318529,0.708039,0.463028,-0.148597,-1.142856,0.569885,-0.911629,-1.312449,-1.145432,1.428424,...,0,0,0,-0.928970,0,0.622652,0,0,0,2355.0
1,0.796515,-0.234062,-0.810597,-0.594176,-0.285041,1.230415,1.212265,1.476947,1.290033,0.816578,...,0,1,0,1.076461,0,-1.606033,0,0,0,4100.0
2,-0.280883,-0.234062,-1.131689,-0.294798,-0.505919,0.492350,0.223076,-1.765727,0.441252,-1.188917,...,0,0,0,1.076461,0,0.622652,0,0,0,2000.0
3,-1.297637,-0.668225,0.252468,-0.578262,0.303904,0.057951,-0.911629,-0.057221,-0.898773,0.068767,...,0,0,0,-0.928970,0,0.622652,0,0,0,500.0
4,0.810146,-0.079438,-0.389553,-0.216467,-0.249732,0.357159,1.212265,1.337478,-1.040187,0.544647,...,0,0,0,1.076461,0,-1.606033,0,0,0,3480.0


In [33]:
data.columns

Index(['TRV.Price', 'tot_bld_annl_rev_amt', 'fnl_prc_pt_fct', 'merge_freq',
       'w2v', 'program_freq', 'prcnt_blg_tiv', 'terr_pctl_bldgfire',
       'comp_freq', 'terr_pctl_liab', 'nbr_comp', 'terr_pctl_bldgcrime',
       'mnln_cd', 'tot_ref_rule_cnt', 'terr_pctl_weather', 'terr_pctl_water',
       'terr_pctl_cntscrime', 'rrv_rt_fct', 'terr_pctl_cntsfire',
       'SUM_BLD_SQFTAGE_bkt_medium', 'Segment_CONDOMINIUMS', 'Segment_OFFICE',
       'zip_freq', 'Region_CENTRAL', 'Segment_RESTAURANTS',
       'Segment_CONTRACTORS', 'State_CT', 'market_sh_low', 'Comp_rev_Hartford',
       'Region_WESTERN', 'State_TN', 'FirstQuoteYear_2016',
       'SUM_BLD_SQFTAGE_bkt_low', 'FirstEffYear_2018', 'UWC_KANSAS CITY',
       'State_VA', 'State_MI', 'Comp_rev_Other', 'Segment_APARTMENTS',
       'Segment_STORE', 'Region_NORTHEAST', 'earlyshop_mth_bkt_0',
       'SPRNKL_IND_bkt', 'market_sh_high', 'State_UT',
       'SUM_BLD_SQFTAGE_bkt_high', 'div_low', 'Segment_BUILDING', 'State_MS',
       'div_no

In [82]:
data.columns[7]
data.terr_pctl_bldgfire.head()

0   -1.312449
1    1.476947
2   -1.765727
3   -0.057221
4    1.337478
Name: terr_pctl_bldgfire, dtype: float64

In [84]:
feat1[0:7, 7]

array([-1.31244949,  1.47694749, -1.7657265 , -0.05722085,  1.33747764,
       -0.37102801, -1.7657265 ])

In [49]:
feat1 = np.asarray(data[['TRV.Price', 'tot_bld_annl_rev_amt', 'fnl_prc_pt_fct', 'merge_freq',
       'w2v', 'program_freq', 'prcnt_blg_tiv', 'terr_pctl_bldgfire',
       'comp_freq', 'terr_pctl_liab', 'nbr_comp', 'terr_pctl_bldgcrime',
       'mnln_cd', 'tot_ref_rule_cnt', 'terr_pctl_weather', 'terr_pctl_water',
       'terr_pctl_cntscrime', 'rrv_rt_fct', 'terr_pctl_cntsfire',
       'SUM_BLD_SQFTAGE_bkt_medium', 'Segment_CONDOMINIUMS', 'Segment_OFFICE',
       'zip_freq', 'Region_CENTRAL', 'Segment_RESTAURANTS',
       'Segment_CONTRACTORS', 'State_CT', 'market_sh_low', 'Comp_rev_Hartford',
       'Region_WESTERN', 'State_TN', 'FirstQuoteYear_2016',
       'SUM_BLD_SQFTAGE_bkt_low', 'FirstEffYear_2018', 'UWC_KANSAS CITY',
       'State_VA', 'State_MI', 'Comp_rev_Other', 'Segment_APARTMENTS',
       'Segment_STORE', 'Region_NORTHEAST', 'earlyshop_mth_bkt_0',
       'SPRNKL_IND_bkt', 'market_sh_high', 'State_UT',
       'SUM_BLD_SQFTAGE_bkt_high', 'div_low', 'Segment_BUILDING', 'State_MS',
       'div_none', 'UWC_CHARLOTTE', 'UWC_DALLAS', 'csf', 'market_sh_med',
       'bld_ind', 'UWC_HARTFORD', 'afi_pol_cnt',
       'program_rev_BUS PAC NAIL SALONS', 'LOC_BLD_AGE_bkt_highest',
       'program_rev_GARAGE PAC GENERAL REPAIRS WIT', 
       ]].head(3000))
#feat1 = feat1.astype(float)

In [50]:
y1 = np.asarray(data[['Competitor.Premium']].head(3000))
#y1 = y1.astype(float)


In [61]:
scalery1 = StandardScaler()
scalery1.fit(y1)
y1 = scalery1.transform(y1)

### Train neural network

In [62]:
use_main_effect_nets = True # toggle this to use "main effect" nets


In [63]:
# Parameters
learning_rate = 0.01
num_epochs = 200
batch_size = 100
display_step = 100
l1_const = 5e-5
num_samples = 3000 #3k datapoints, split 1/3-1/3-1/3

# Network Parameters
n_hidden_1 = 140 # 1st layer number of neurons
n_hidden_2 = 100 # 2nd layer number of neurons
n_hidden_3 = 60 # 3rd "
n_hidden_4 = 20 # 4th "
n_hidden_uni = 10
num_input = 60 # simple synthetic example input dimension
num_output = 1 # regression or classification output dimension

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_output])

tf.set_random_seed(81)
np.random.seed(81)


In [64]:

tr_x1, va_x1, te_x1 = feat1[:1000], feat1[1000:2000], feat1[2000:]
tr_y1, va_y1, te_y1 = y1[:1000], y1[1000:2000], y1[2000:]


In [65]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()

scaler_x.fit(tr_x1)
scaler_y.fit(tr_y1)

tr_x, va_x, te_x = scaler_x.transform(tr_x1), scaler_x.transform(va_x1), scaler_x.transform(te_x1)

tr_y, va_y, te_y = scaler_y.transform(tr_y1), scaler_y.transform(va_y1), scaler_y.transform(te_y1)

tr_size = tr_x.shape[0]
print(tr_size)


1000


In [66]:
# access weights & biases
weights = {
    'h1': tf.Variable(tf.truncated_normal([num_input, n_hidden_1], 0, 0.1)),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], 0, 0.1)),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], 0, 0.1)),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4, num_output], 0, 0.1))
}

biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1], 0, 0.1)),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2], 0, 0.1)),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3], 0, 0.1)),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.truncated_normal([num_output], 0, 0.1))
}

def get_weights_uninet():
    weights = {
        'h1': tf.Variable(tf.truncated_normal([1, n_hidden_uni], 0, 0.1)),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_uni, n_hidden_uni], 0, 0.1)),
        'h3': tf.Variable(tf.truncated_normal([n_hidden_uni, n_hidden_uni], 0, 0.1)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_uni, num_output], 0, 0.1))
    }
    return weights

def get_biases_uninet():
    biases = {
        'b1': tf.Variable(tf.truncated_normal([n_hidden_uni], 0, 0.1)),
        'b2': tf.Variable(tf.truncated_normal([n_hidden_uni], 0, 0.1)),
        'b3': tf.Variable(tf.truncated_normal([n_hidden_uni], 0, 0.1))
    }
    return biases


In [67]:
#Create model
def normal_neural_net(x, weights, biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))
    layer_4 = tf.nn.relu(tf.add(tf.matmul(layer_3, weights['h4']), biases['b4']))    
    out_layer = tf.nn.sigmoid(tf.matmul(layer_4, weights['out']) + biases['out'])
    return out_layer

def main_effect_net(x, weights, biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))    
    out_layer = tf.nn.sigmoid(tf.matmul(layer_3, weights['out']))
    return out_layer

# L1 regularizer
def l1_norm(a): return tf.reduce_sum(tf.abs(a))


In [68]:
# Construct model
net = normal_neural_net(X, weights, biases)

if use_main_effect_nets:  
    me_nets = []
    for x_i in range(num_input):
        me_net = main_effect_net(tf.expand_dims(X[:,x_i],1), get_weights_uninet(), get_biases_uninet())
        me_nets.append(me_net)
    net = net + sum(me_nets)

# Define optimizer
loss_op = tf.losses.mean_squared_error(labels=Y, predictions=net)
# loss_op = tf.sigmoid_cross_entropy_with_logits(labels=Y,logits=net) # use this in the case of binary classification
sum_l1 = tf.reduce_sum([l1_norm(weights[k]) for k in weights])
loss_w_reg_op = loss_op + l1_const*sum_l1 

batch = tf.Variable(0)
decaying_learning_rate = tf.train.exponential_decay(learning_rate, batch*batch_size, tr_size, 0.95, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate=decaying_learning_rate).minimize(loss_w_reg_op, global_step=batch)


In [69]:
init = tf.global_variables_initializer()
n_batches = tr_size//batch_size
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.25
# config.gpu_options.allow_growth = True
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Initialized')

for epoch in range(num_epochs):

    batch_order = list(range(n_batches))
    np.random.shuffle(batch_order)

    for i in batch_order:
        batch_x = tr_x1[i*batch_size:(i+1)*batch_size]
        batch_y = tr_y1[i*batch_size:(i+1)*batch_size]
        _, lr = sess.run([optimizer,decaying_learning_rate], feed_dict={X:batch_x, Y:batch_y})

    if (epoch+1) % 50 == 0:
        tr_mse = sess.run(loss_op, feed_dict={X:tr_x1, Y:tr_y1})
        va_mse = sess.run(loss_op, feed_dict={X:va_x1, Y:va_y1})
        te_mse = sess.run(loss_op, feed_dict={X:te_x1, Y:te_y1})
        print('Epoch', epoch+1)
        print('\t','train rmse', math.sqrt(tr_mse), 'val rmse', math.sqrt(va_mse), 'test rmse', math.sqrt(te_mse))
        print('\t','learning rate', lr)
        
print('done')


Initialized
Epoch 50
	 train rmse 1.120186415298193 val rmse 1.1126489282427932 test rmse 1.196559778839548
	 learning rate 0.00080994656
Epoch 100
	 train rmse 1.1167470423870833 val rmse 1.109081551868375 test rmse 1.1937554384277451
	 learning rate 6.232128e-05
Epoch 150
	 train rmse 1.116404919526767 val rmse 1.1087269031652056 test rmse 1.1934767445287047
	 learning rate 4.7953063e-06
Epoch 200
	 train rmse 1.1163703223948307 val rmse 1.108690829929164 test rmse 1.1934485269513047
	 learning rate 3.689745e-07
done


### Interprete weights

In [70]:
def preprocess_weights(w_dict):
    hidden_layers = [int(layer[1:]) for layer in w_dict.keys() if layer.startswith('h')]
    output_h = ['h' + str(x) for x in range(max(hidden_layers),1,-1)]
    w_agg = np.abs(w_dict['out'])
    w_h1 = np.abs(w_dict['h1'])

    for h in output_h:
        w_agg = np.matmul( np.abs(w_dict[h]), w_agg)

    return w_h1, w_agg 

def get_interaction_ranking(w_dict):
    xdim = w_dict['h1'].shape[0]
    w_h1, w_agg = preprocess_weights(w_dict)
        
    # rank interactions
    interaction_strengths = dict()

    for i in range(len(w_agg)):
        sorted_fweights = sorted(enumerate(w_h1[:,i]), key=lambda x:x[1], reverse = True)
        interaction_candidate = []
        weight_list = []       
        for j in range(len(w_h1)):
            bisect.insort(interaction_candidate, sorted_fweights[j][0]+1)
            weight_list.append(sorted_fweights[j][1])
            if len(interaction_candidate) == 1:
                continue
            interaction_tup = tuple(interaction_candidate)
            if interaction_tup not in interaction_strengths:
                interaction_strengths[interaction_tup] = 0
            inter_agg = min(weight_list)      
            interaction_strengths[interaction_tup] += np.abs(inter_agg*np.sum(w_agg[i]))
        
    interaction_sorted = sorted(interaction_strengths.items(), key=operator.itemgetter(1), reverse=True)

    # forward prune the ranking of redundant interactions
    interaction_ranking_pruned = []
    existing_largest = []
    for i, inter in enumerate(interaction_sorted):
        if len(interaction_ranking_pruned) > 20000: break
        skip = False
        indices_to_remove = set()
        for inter2_i, inter2 in enumerate(existing_largest):
            # if this is not the existing largest
            if set(inter[0]) < set(inter2[0]):
                skip = True
                break
            # if this is larger, then need to recall this index later to remove it from existing_largest
            if set(inter[0]) > set(inter2[0]):
                indices_to_remove.add(inter2_i)
        if skip:
            assert len(indices_to_remove) == 0
            continue
        prevlen = len(existing_largest)
        existing_largest[:] = [el for el_i, el in enumerate(existing_largest) if el_i not in indices_to_remove]
        existing_largest.append(inter)
        interaction_ranking_pruned.append((inter[0], inter[1]))

        curlen = len(existing_largest)

    return interaction_ranking_pruned

def get_pairwise_ranking(w_dict):
    xdim = w_dict['h1'].shape[0]
    w_h1, w_agg = preprocess_weights(w_dict)

    input_range = range(1,xdim+1)
    pairs = [(xa,yb) for xa in input_range for yb in input_range if xa != yb]
    for entry in pairs:
        if (entry[1], entry[0]) in pairs:
            pairs.remove((entry[1],entry[0]))

    pairwise_strengths = []
    for pair in pairs:
        a = pair[0]
        b = pair[1]
        wa = w_h1[a-1].reshape(w_h1[a-1].shape[0],1)
        wb = w_h1[b-1].reshape(w_h1[b-1].shape[0],1)
        wz = np.abs(np.minimum(wa , wb))*w_agg
        cab = np.sum(np.abs(wz))
        pairwise_strengths.append((pair, cab))
#     list(zip(pairs, pairwise_strengths))

    pairwise_ranking = sorted(pairwise_strengths,key=operator.itemgetter(1), reverse=True)

    return pairwise_ranking


In [71]:

w_dict = sess.run(weights)


In [72]:
# Variable-Order Interaction Ranking
get_interaction_ranking(w_dict)


[((7, 16), 2.332972466945648),
 ((5, 46), 2.205996036529541),
 ((17, 20), 2.0445556640625),
 ((7, 13), 2.035025894641876),
 ((55, 57), 2.0244052708148956),
 ((9, 55), 1.832545906305313),
 ((17, 53), 1.7864608764648438),
 ((53, 54), 1.6404598951339722),
 ((33, 50), 1.6210207343101501),
 ((7, 10, 16), 1.472190260887146),
 ((6, 53, 54), 1.4535362720489502),
 ((42, 50), 1.4531493186950684),
 ((7, 10, 16, 20), 1.452885389328003),
 ((42, 55), 1.4420205354690552),
 ((41, 42, 55), 1.4390422105789185),
 ((2, 44), 1.4328253269195557),
 ((2, 40, 44), 1.424058198928833),
 ((2, 4, 40, 44), 1.4215259552001953),
 ((14, 30), 1.4095714092254639),
 ((6, 10, 53, 54), 1.4056386947631836),
 ((7, 41, 42, 55), 1.4033148288726807),
 ((7, 28, 41, 42, 55), 1.3882383108139038),
 ((23, 39), 1.3861080408096313),
 ((7, 12, 28, 41, 42, 55), 1.380401611328125),
 ((2, 7, 12, 28, 41, 42, 55), 1.3739482164382935),
 ((8, 40), 1.3687330484390259),
 ((1, 18), 1.3593473434448242),
 ((23, 39, 53), 1.3565245866775513),
 ((2, 

In [75]:
feat1[7]

array([ 0.12215901, -1.10238849,  0.46302803, -0.5623485 , -0.15686207,
       -0.40263818,  1.19953537,  0.88420063,  1.2900329 , -0.06719916,
       -0.33707252, -0.62565368,  0.        ,  0.67870008, -1.25810593,
       -1.47231023,  0.48698887, -1.29758582, -0.89195791,  0.        ,
        0.        ,  0.        , -0.60891822,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.07646085,
        0.        , -1.60603326,  0.        ,  0.        ,  0.        ])

In [73]:
# Pairwise Interaction Ranking
get_pairwise_ranking(w_dict)


[((9, 55), 36.600594),
 ((7, 9), 35.01552),
 ((7, 11), 34.883446),
 ((7, 55), 34.726517),
 ((7, 50), 34.652946),
 ((3, 7), 34.45057),
 ((7, 53), 33.85553),
 ((9, 17), 33.848434),
 ((18, 55), 33.825027),
 ((17, 57), 33.80439),
 ((7, 20), 33.761963),
 ((15, 20), 33.37201),
 ((8, 55), 33.356796),
 ((9, 20), 33.340454),
 ((42, 55), 33.29687),
 ((3, 9), 33.098885),
 ((7, 8), 33.07682),
 ((20, 42), 33.006664),
 ((8, 9), 32.762447),
 ((7, 57), 32.69879),
 ((7, 18), 32.678284),
 ((11, 15), 32.635075),
 ((7, 42), 32.600704),
 ((7, 15), 32.44691),
 ((15, 55), 32.32428),
 ((9, 12), 32.094707),
 ((12, 53), 32.094025),
 ((18, 20), 32.049225),
 ((7, 17), 32.001736),
 ((17, 20), 31.891613),
 ((7, 12), 31.878214),
 ((7, 16), 31.74615),
 ((11, 20), 31.739088),
 ((9, 57), 31.718441),
 ((20, 57), 31.712059),
 ((8, 57), 31.584026),
 ((3, 20), 31.57066),
 ((8, 17), 31.564768),
 ((19, 55), 31.526634),
 ((15, 18), 31.477734),
 ((10, 20), 31.366787),
 ((17, 50), 31.274403),
 ((11, 57), 31.234268),
 ((8, 42), 